This notebook is based on the [send-receive-pulse demo](../qick_demos/00_Send_receive_pulse.ipynb).

In [ ]:
# Import the QICK drivers and auxiliary libraries
from qick import *
%pylab inline

In [ ]:
import os

# Custom bistream
FERMILAB_BIT = 'qick_216.bit'

# Normalize path
FERMILAB_BIT = os.path.normpath(os.getcwd() + '/' + FERMILAB_BIT)

print('Custom bitsream:', FERMILAB_BIT)

In [ ]:
# Load bitstream with custom overlay
soc = QickSoc(bitfile=FERMILAB_BIT)
# Since we're running locally on the QICK, we don't need a separate QickConfig object.
# If running remotely, you could generate a QickConfig from the QickSoc:
#     soccfg = QickConfig(soc.get_cfg())
# or save the config to file, and load it later:
#     with open("qick_config.json", "w") as f:
#         f.write(soc.dump_cfg())
#     soccfg = QickConfig("qick_config.json")
soccfg = soc
print(soccfg)

### Hardware Configuration

generator channel 6   : DAC 229 CH3  <-> Readout channel 0 : ADC 224 CH0

## Things to know when writing programs

### How to avoid timing problems
Timed instructions are executed in two steps: first when the control core of the tProcessor pushes the instruction into the timed queue, and second when the instruction pops off the timed queue.
* The control core executes all instructions in order and (except wait/waiti) as quickly as possible. If you have a non-timed instruction with external effects, it will generally execute before the timed instructions before and after it. Specifically, programs typically use memwi to update a data counter, and then read that counter to know how many data samples to get from the buffers - if the memwi happens before the readout is triggered or completed, you will read invalid data.
* If the instruction is pushed later than the specified time, it will execute later than specified - this can lead to variations in the relative timing of pulses and readout windows. You are responsible for keeping the control core running ahead of the instruction times.

Things you should do in your programs:
* Your initialization should contain a synci instruction which gives the tProcessor time to get ahead of the clock.
* Put a `sync_all()` somewhere in your loop body, probably at the end. This ensures that you don't have pulses or readouts that exceed the loop length.
* Put a waiti instruction after the last readout in your loop body, to make sure the tProcessor doesn't prematurely update the data counter.
* Follow the waiti with a sync that exceeds the waiti, to allow the tProcessor to get ahead of the clock.

The last three points are automatically addressed if the last timed instruction in your loop body is a `measure()` instruction that specifies `wait=True` and a nonzero `syncdelay`.

### How to ensure frequency matching
DACs and ADCs have different frequency units, and the frequencies used in the two systems must be exactly equal. If they are not, there will be a small difference between the upconversion and downconversion frequencies - this will manifest as a sliding phase, and so you will not see a consistent phase between acquisitions.

There are two ways to ensure frequency matching:
* When converting a frequency to an integer value (often with `freq2reg()` or `declare_readout()`), specify not only the channel you are configuring, but the channel you want to be frequency-matched to.
* Before doing any conversion, round the frequency to the closest frequency that is valid on both channels using `soccfg.adcfreq(f, gen_ch, ro_ch)`.

Frequency-matching makes your frequency resolution worse, since the smallest possible frequency step is now the LCM of the two channels' frequency steps. Usually this doesn't matter - O(10 Hz) resolution is ample for most applications - but you can disable frequency-matching by specifying None as the other channel.

You may have a DAC channel that does not itself drive any ADC channels, but needs to be phase-locked to a DAC channel that does. In this case you will want to frequency-match both DAC channels to that ADC, otherwise you will have a sliding phase between the two DAC channels.

Since all DAC and ADC channels are the same in the standard QICK firmware, it's OK to just use ch 0 for the matched channel. But it's a good habit to do this consistently correctly, and that is the approach here.

### Clocks and durations
Time durations are generally specified in units of clock cycles. The relevant clocks are the tProcessor clock and the fabric clocks of the DACs and ADCs. In general these can all be different (and can even vary among DACs), though the standard ZCU111 firmware uses the same clock in all three places.

For convenience, the `us2cycles()` and `cycles2us()` methods will convert between floating-point times and integer cycles. You should be careful to specify which clock you are using, and set the appropriate parameter in `us2cycles()`:
* Pulse parameters (the `length` parameter to `set_pulse_registers()`, the `length` and `sigma` parameters to `add_gauss`) use the DAC clock and you should specify `gen_ch`.
* Readout parameters (the `length` parameter to `declare_readout()`) use the ADC clock and you should specify `ro_ch`.
* All other values will use the tProc clock. This includes sync and wait commands, and any sort of delay (`t`, `adc_trig_offset`). No special parameter needed.

### How to play pulses
There are three steps to playing a pulse:

#### Loading a waveform
(You skip this step for rectangular "const" pulses, which have no envelope.)

Each signal generator has an internal waveform memory, which stores the I/Q data for the pulse envelope. Multiple waveforms can be stored in the same signal generator, and a single waveform can be used for different pulses (e.g. a Gaussian waveform can be used for Gaussian pulses and the ramp-up/ramp-down of flat-top pulses with different flat-top duration, each with its own gain and carrier frequency).

`add_pulse(ch, name, idata, qdata)` writes an arbitrary waveform to the specified channel's waveform memory. 
`add_gauss(ch, name, length, sigma)`, `add_triangle(ch, name, length)`, and `add_DRAG(ch, name, length, sigma, delta, alpha)` write commonly-used standard pulse waveforms, with duration units of fabric clock cycles. The name is used in the next step.

#### Setting registers
There are a lot of parameters that need to be specified when playing a pulse - more than can be specified inline in a tProcessor instruction. So all the parameters must be written to registers first, and when we fire the pulse we just tell the tProcessor which registers to read.

`set_pulse_registers()` writes the settings for a pulse to registers. All arguments to this method must be integers in the native units of the signal generator. This can happen immediately before you fire the pulse, but if a signal generator is only used for one type of pulse you will save time for the tProcessor by setting registers in initialization, before the program loop.

If you want to modify pulse parameters on the fly (for example, you might want to sweep the frequency of the qubit drive pulse), you will set the registers in two steps (you can see an example in the demo 02_Sweeping_variables):
* First, use set_pulse_registers() to write initial values for all parameters. You could do this in initialization, or right before the following step.
* Second, overwrite the register(s) you want to update. You need to get the page and address of the register using ch_page(ch) and sreg(ch, name). Then you can use assembly instructions to change the value of that register.

#### Firing the pulse
`pulse(ch, t)` fires a pulse on the specified channel at the specified time, using whatever values are loaded in the registers.

Often you will want to trigger the readout at the same time: `measure()` is a wrapper around `trigger()` and `pulse()`, and that's what is used in this demo.

In [ ]:
class LoopbackProgram(AveragerProgram):
    def initialize(self):
        cfg=self.cfg   
        res_ch = cfg["res_ch"]

        # set the nyquist zone
        self.declare_gen(ch=cfg["res_ch"], nqz=1)
        
        # configure the readout lengths and downconversion frequencies (ensuring it is an available DAC frequency)
        for ch in cfg["ro_chs"]:
            self.declare_readout(ch=ch, length=self.cfg["readout_length"],
                                 freq=self.cfg["pulse_freq"], gen_ch=cfg["res_ch"])

        # convert frequency to DAC frequency (ensuring it is an available ADC frequency)
        freq = self.freq2reg(cfg["pulse_freq"],gen_ch=res_ch, ro_ch=cfg["ro_chs"][0])
        phase = self.deg2reg(cfg["res_phase"], gen_ch=res_ch)
        gain = cfg["pulse_gain"]
        self.default_pulse_registers(ch=res_ch, freq=freq, phase=phase, gain=gain)

        style=self.cfg["pulse_style"]

        if style in ["flat_top","arb"]:
            sigma = cfg["sigma"]
            self.add_gauss(ch=res_ch, name="measure", sigma=sigma, length=sigma*5)
            
        if style == "const":
            self.set_pulse_registers(ch=res_ch, style=style, length=cfg["length"])
        elif style == "flat_top":
            # The first half of the waveform ramps up the pulse, the second half ramps down the pulse
            self.set_pulse_registers(ch=res_ch, style=style, waveform="measure", length=cfg["length"])
        elif style == "arb":
            self.set_pulse_registers(ch=res_ch, style=style, waveform="measure")
        
        self.synci(200)  # give processor some time to configure pulses
    
    def body(self):
        # fire the pulse
        # trigger all declared ADCs
        # pulse PMOD0_0 for a scope trigger
        # pause the tProc until readout is done
        # increment the time counter to give some time before the next measurement
        # (the syncdelay also lets the tProc get back ahead of the clock)
        self.measure(pulse_ch=self.cfg["res_ch"], 
                     adcs=self.ro_chs,
                     pins=[0], 
                     adc_trig_offset=self.cfg["adc_trig_offset"],
                     wait=True,
                     syncdelay=self.us2cycles(self.cfg["relax_delay"]))
        
        # equivalent to the following:
#         self.trigger(adcs=self.ro_chs,
#                      pins=[0], 
#                      adc_trig_offset=self.cfg["adc_trig_offset"])
#         self.pulse(ch=self.cfg["res_ch"])
#         self.wait_all()
#         self.sync_all(self.us2cycles(self.cfg["relax_delay"]))

### Send/receive a pulse with <code> pulse_style </code> = <code> const </code>

In [ ]:
config={"res_ch":6, # --Fixed
        "ro_chs":[0], # --Fixed
        "reps":1, # --Fixed
        "relax_delay":1.0, # --us
        "res_phase":0, # --degrees
        "pulse_style": "const", # --Fixed
        
        "length":20, # [Clock ticks]
        # Try varying length from 10-100 clock ticks
        
        "readout_length":100, # [Clock ticks]
        # Try varying readout_length from 50-1000 clock ticks

        "pulse_gain":6000, # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units

        "pulse_freq": 100, # [MHz]
        # In this program the signal is up and downconverted digitally so you won't see any frequency
        # components in the I/Q traces below. But since the signal gain depends on frequency, 
        # if you lower pulse_freq you will see an increased gain.

        "adc_trig_offset": 100, # [Clock ticks]
        # Try varying adc_trig_offset from 100 to 220 clock ticks

        "soft_avgs":1
        # Try varying soft_avgs from 1 to 200 averages

       }

###################
# Try it yourself !
###################

prog =LoopbackProgram(soccfg, config)
iq_list = prog.acquire_decimated(soc, load_pulses=True, progress=True, debug=False)

In [ ]:
# Plot results.
plt.figure(1)
for ii, iq in enumerate(iq_list):
    plt.plot(iq[0], label="I value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(iq[1], label="Q value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(np.abs(iq[0]+1j*iq[1]), label="mag, ADC %d"%(config['ro_chs'][ii]))
plt.ylabel("a.u.")
plt.xlabel("Clock ticks")
plt.title("Averages = " + str(config["soft_avgs"]))
plt.legend()
plt.savefig("images/Send_recieve_pulse_const.pdf", dpi=350)

### Rep-to-rep consistency
In this notebook we mostly use decimated readout (we acquire a full waveform, not just a single accumulated value). To avoid exhausting the waveform buffer, we usually run with reps=1 (the tProcessor program only fires+reads one pulse, and we run the program `soft_avgs` times).

However, it's important to check that if you run a tProcessor loop with reps>1, the pulse looks the same in each iteration of the loop. So let's do a decimated readout with multiple reps; the 1024-sample buffer will allow for 10 reps of 100 samples each.

Try changing the relax_delay to 0. You will see that the first rep appears in the same place, but the others are delayed by a bit because the tProcessor didn't have enough time between waiting for the measurement in one rep and firing the pulse in the next rep.

### Multiple triggers
You can send multiple triggers inside a loop. The `acquire()` and `acquire_decimated()` methods take a `readouts_per_experiment` parameter which must equal the number of triggers.

Let's check the rep-to-rep consistency as well as the consistency between the first and second trigger.

### Multiple pulses

Here's an example of playing multiple pulses from the same generator, with a 90-degree shift in the carrier phase between the two pulses.

### Send/receive a pulse with <code> pulse_style </code> = <code> flat_top </code>

### Send/receive a pulse with <code> pulse_style </code> = <code> arb </code>

In [ ]:
config={"res_ch":6, # --Fixed
        "ro_chs":[0], # --Fixed
        "reps":1, # --Fixed
        "relax_delay":1.0, # --us
        "res_phase":0, # --degrees
        "pulse_style": "arb", # --Fixed
        
        "sigma": 30, # [Clock ticks]
        # Try varying sigma from 10-50 clock ticks
        
        "readout_length":300, # [Clock ticks]
        # Try varying readout_length from 50-1000 clock ticks

        "pulse_gain":5000, # [DAC units]
        # Try varying pulse_gain from 500 to 30000 DAC units

        "pulse_freq": 100, # [MHz]
        # In this program the signal is up and downconverted digitally so you won't see any frequency
        # components in the I/Q traces below. But since the signal gain depends on frequency, 
        # if you lower pulse_freq you will see an increased gain.

        "adc_trig_offset": 100, # [Clock ticks]
        # Try varying adc_trig_offset from 100 to 220 clock ticks

        "soft_avgs":100
        # Try varying soft_avgs from 1 to 200 averages

       }

###################
# Try it yourself !
###################

prog =LoopbackProgram(soccfg, config)
iq_list = prog.acquire_decimated(soc, load_pulses=True, progress=True, debug=False)


In [ ]:
# Plot results.
plt.figure(1)
for ii, iq in enumerate(iq_list):
    plt.plot(iq[0], label="I value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(iq[1], label="Q value, ADC %d"%(config['ro_chs'][ii]))
    plt.plot(np.abs(iq[0]+1j*iq[1]), label="mag, ADC %d"%(config['ro_chs'][ii]))
plt.ylabel("a.u.")
plt.xlabel("Clock ticks")
plt.title("Averages = " + str(config["soft_avgs"]))
plt.legend()
plt.savefig("images/Send_recieve_pulse_arb.pdf", dpi=350)

### Compare the two main ways of acquiring data: <code>prog.acquire_decimated</code> and <code>prog.acquire</code>

In the previous two demonstrations we used <code>prog.acquire_decimated</code> which uses the QICK decimated buffer (acquiring a whole time trace of data for every measurement shot. The QICK accumulated buffer is used in <code>prog.acquire</code>, which acquires a single I/Q data point per measurement shot- the average of the I/Q values in the decimated buffer. In qubit experiments we will be mainly using <code>prog.acquire</code> since for every shot we only need one I/Q value to assess the state of the qubit. So let's verify that <code>prog.acquire_decimated</code> and <code>prog.acquire</code> produce similar results (to within +/- 1 DAC units).

In [ ]:
# First, lets collect the results with the decimated buffer as we did before.
config={"res_ch":6, # --Fixed
        "ro_chs":[0], # --Fixed
        "reps":1, # --Fixed
        "relax_delay":1, # --Fixed
        "res_phase":0, # --Fixed
        "pulse_style": "const", # --Fixed
        "length":20, # [Clock ticks]        
        "readout_length":200, # [Clock ticks]
        "pulse_gain":10000, # [DAC units]
        "pulse_freq": 100, # [MHz]
        "adc_trig_offset": 100, # [Clock ticks]
        "soft_avgs":100
       }

prog =LoopbackProgram(soccfg, config)
iq_list = prog.acquire_decimated(soc, load_pulses=True, progress=True, debug=False)


In [ ]:
print("I value; ADC 0; Decimated buffer: ",  np.mean(iq_list[0][0]))
print("Q value; ADC 0; Decimated buffer: ", np.mean(iq_list[0][1]))


In [ ]:
# Now, lets collect the results with the accumulated buffer. 
config["reps"] = 100; # Set reps equal to soft_avgs in the prior acquisition method

prog =LoopbackProgram(soccfg, config)
avgi, avgq = prog.acquire(soc, load_pulses=True)

In [ ]:
print("I value; ADC 0; Accumulated buffer: ", avgi[0])
print("Q value; ADC 0; Accumulated buffer: ", avgq[0])


### Printing the program
It's sometimes useful to print the program in ASM format, to get a feeling for what's going on.

In [ ]:
print(prog)